In [ ]:
pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 50.3 MB/s eta 0:00:00


In [ ]:
import json
import os
import re
from underthesea import word_tokenize

No module named 'fasttext'


# Mục mới

In [ ]:
def load_all_data(data_path):
    all_data = []
    for filename in os.listdir(data_path):
        if filename.endswith('.json'):
            file_path = os.path.join(data_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_data.extend(data['data'])
    return {'data': all_data}

In [ ]:

def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [ ]:
def preprocess_data(data):
    contexts = []
    questions = []
    answers = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = clean_text(paragraph['context'])
            for qa in paragraph['qas']:
                question = clean_text(qa['question'])
                is_impossible = qa.get('is_impossible', False)  #Do ngữ liệu nhiều chỗ define thiếu is_impossible nên sẽ mặc định là False
                if not is_impossible:
                    for answer in qa['answers']:
                        answer_text = clean_text(answer['text'])
                        answer_start = answer['answer_start']
                        contexts.append(context)
                        questions.append(question)
                        answers.append({
                            'text': answer_text,
                            'start': answer_start
                        })
    return contexts, questions, answers

In [ ]:
def tokenize_texts(texts):
    return [word_tokenize(text, format="text") for text in texts]

In [ ]:
data_path = '/content/data'
squad_data = load_all_data(data_path)
contexts, questions, answers = preprocess_data(squad_data)
tokenized_contexts = tokenize_texts(contexts)
tokenized_questions = tokenize_texts(questions)

In [ ]:
tokenized_contexts

In [ ]:
pip install gensim

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [ ]:
# Tokenize ngữ liệu
split_context_pre_train = [context.split() for context in tokenized_contexts]
split_question_pre_train = [simple_preprocess(question) for question in tokenized_questions]

In [ ]:
# Tạo mô hình Word2Vec
model_word2vec = Word2Vec(vector_size=100, window=10, min_count=1, sg=1, workers=4)

In [ ]:
# Xây dựng từ điển
model_word2vec.build_vocab(split_question_pre_train)

In [ ]:
model_word2vec.train(split_question_pre_train, total_examples=model_word2vec.corpus_count, epochs=100)

(1651025, 3103200)

In [ ]:
# Lưu mô hình Word2Vec đã huấn luyện
model_word2vec.save("/content/word2vec.model")

In [ ]:
vector = model_word2vec.wv['chương']

In [ ]:
vector

In [ ]:
similarWord = model_word2vec.wv.most_similar('chương', topn=5)
print(similarWord)

[('bộ_luật', 0.6755515933036804), ('chính', 0.6376974582672119), ('việt_nam', 0.5604852437973022), ('v', 0.5576168298721313), ('bao_gồm', 0.543369710445404)]


In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

# Tạo BM25 model
bm25 = BM25Okapi(split_question_pre_train)

In [ ]:
questionBm25 = "Lương ngoài giờ tính như thế nào"
tokenized_query = questionBm25.split(" ")


In [ ]:
# Tính điểm số BM25
bm25_scores = bm25.get_scores(tokenized_query)

In [ ]:
import numpy as np

# Xếp hạng các câu hỏi dựa trên điểm số
top_n = np.argsort(bm25_scores)[::-1][:10]

# Hiển thị các câu hỏi có điểm số cao nhất
print("Câu hỏi người dùng:", query)
print("Các câu hỏi có điểm số cao nhất:")
for index in top_n:
    print(f"Điểm số: {bm25_scores[index]:.4f} - Câu hỏi: {split_question_pre_train[index]}")


Câu hỏi người dùng: Lương ngoài giờ tính như thế nào
Các câu hỏi có điểm số cao nhất:
Điểm số: 11.3968 - Câu hỏi: ['thời_gian', 'làm_việc', 'ban_đêm', 'được', 'tính', 'từ', 'mấy', 'giờ', 'đến', 'mấy', 'giờ']
Điểm số: 11.3861 - Câu hỏi: ['tiền_lương', 'làm', 'thêm', 'giờ', 'được', 'tính', 'như_thế_nào']
Điểm số: 11.3861 - Câu hỏi: ['tiền_lương', 'làm', 'thêm', 'giờ', 'được', 'tính', 'như_thế_nào']
Điểm số: 10.4546 - Câu hỏi: ['thời', 'giờ', 'làm_việc', 'được', 'tính', 'vào', 'thời_giờ', 'làm_việc', 'được', 'hưởng', 'lương', 'bao_gồm', 'những', 'khoảng', 'thời_gian', 'nào']
Điểm số: 7.5303 - Câu hỏi: ['chương_trình', 'đào_tạo', 'nghề', 'có_thể', 'được', 'tổ_chức', 'ngoài', 'nước', 'không']
Điểm số: 7.2347 - Câu hỏi: ['người', 'lao_động', 'có', 'quyền', 'tham_gia', 'các', 'khóa_học', 'ngoài', 'doanh_nghiệp', 'không']
Điểm số: 7.0543 - Câu hỏi: ['lao_động', 'đặc_thù', 'có', 'quyền', 'từ_chối', 'làm', 'thêm', 'giờ', 'trong', 'trường_hợp', 'nào']
Điểm số: 6.6262 - Câu hỏi: ['tiền_lương', 'ng

In [ ]:
def get_avg_word2vec_vector(question):
    words = simple_preprocess(question)
    vector = np.mean([model_word2vec.wv[word] for word in words if word in model_word2vec.wv], axis=0)
    return vector

In [ ]:
def get_word2vec_scores(user_question, questions):
    query_vector = get_avg_word2vec_vector(user_question)
    scores = []
    for question in questions:
        question_vector = get_avg_word2vec_vector(question)
        score = np.dot(query_vector, question_vector)
        scores.append(score)
    return np.array(scores)

In [ ]:
def find_best_matching_question(user_question, questions):
    tokenized_query = simple_preprocess(user_question)
    bm25_scores = bm25.get_scores(tokenized_query)
    word2vec_scores = get_word2vec_scores(user_question, questions)

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / (np.max(word2vec_scores) - np.min(word2vec_scores))

    # Combine scores with equal weight
    combined_scores = bm25_scores + word2vec_scores

    best_idx = np.argmax(combined_scores)
    best_question = questions[best_idx]
    return best_question

In [ ]:
input_question = "Lương ngoài giờ tính như thế nào"
best_question = find_best_matching_question(input_question, questions)

print(f"User Question: {input_question}")
print(f"Best matching question: {best_question}")

User Question: Lương ngoài giờ tính như thế nào
Best matching question: tiền lương làm thêm giờ được tính như thế nào
